In [1]:
import datetime
import numpy as np
import pandas as pd
import requests
import random
import time
import re
import jieba as jb
from bs4 import BeautifulSoup
import bs4

In [2]:
def get_ppt_page(url):
    # 紀錄cookies 是否年滿18歲
    resp = requests.get(
        url=url,
        cookies={'over18': '1'}  
    )
    if resp.status_code != 200:
        print('Invalid url:', resp.url)
        return None
    else:
        return resp.text

In [3]:
def get_content(article_url):
    # 文章連結
    # URL = "https://www.ptt.cc/bbs/Gossiping/M.1639675838.A.A33.html"
    # 設定Header與Cookie
    my_headers = {'cookie': 'over18=1;'}
    # 發送get 請求 到 ptt 八卦版
    response = requests.get(article_url, headers = my_headers)


    #  把網頁程式碼(HTML) 丟入 bs4模組分析
    soup = bs4.BeautifulSoup(response.text,"html.parser")

    ## PTT 上方4個欄位
    header = soup.find_all('span','article-meta-value')


    ## 查找所有html 元素 抓出內容
    main_container = soup.find(id='main-container')
    # 把所有文字都抓出來
    all_text = main_container.text
    # 把整個內容切割透過 "-- " 切割成2個陣列
    pre_text = all_text.split('--')[0]
        
    # 把每段文字 根據 '\n' 切開
    texts = pre_text.split('\n')
    # 如果你爬多篇你會發現 
    contents = texts[2:]
    # 內容
    content = '\n'.join(contents)

    # 顯示
    #print('內容: ', content)
    return content

In [4]:
def get_pageinfo(num, resdata, domain_url, pptdata, date_range):
    soup = BeautifulSoup(resdata, 'html5lib')
    # 取得上一頁按鈕
    paging_div = soup.find('div', 'btn-group btn-group-paging')
    prev_url = paging_div.find_all('a')[1]['href']
    
    # 儲存取得的文章資料<div class="r-ent"></div>
    date_divs = soup.find_all('div', 'r-ent')
    
    whether_prev = True
    for k in date_divs:
        # 判斷文章是否在一週內發佈
        week_date = k.find('div', 'date').text.strip() in date_range
        
        # 代表最早的日期，就不符合條件，所以不用下一頁
        # 因為第一頁涵蓋熱門文章，因此日期並非按照順序，所以從第二頁開始判斷
        if num >= 1:
            if week_date == False:
                whether_prev = False

        # 一周內發布       
        if week_date:
            # 發布日期
            post_date = k.find('div', 'date').text.strip()
            #print("發布日期",post_date)
            
            # 推文數
            push_count = k.find('div', 'nrec').text
            push_num = 0
            if push_count:
                try:
                    push_num = int(push_count)  
                except ValueError:
                    # 若轉換失敗，可能是'爆'或 'X1', 'X2'
                    if push_count == '爆':
                        push_num = 100
                    elif push_count.startswith('X'):
                        push_num = -100
                    else:
                        push_num = 0
            #print("推文數",push_num)
            
            # 有超連結，代表文章存在
            if k.find('a'): 
                # 文章標題
                title = k.find('a').text
                #print("標題",title)
                # 文章連結
                href = k.find('a')['href']
                #print("標題連結",domain_url+href)
                # 文章內文
                content = get_content(domain_url+href)

                # 儲存資料
                pptdata.append({
                    'date': post_date,
                    'push_num': push_num,
                    'title': title,
                    'content': content,
                    'href': domain_url+href
                })

        
    
    # 每做完一頁，num+1
    num += 1
    
    
    return prev_url, pptdata, whether_prev, num

In [5]:
if __name__ == '__main__':
    # *** 選擇八卦版 ***
    domain_url = 'https://www.ptt.cc'
    Gossiping = '/bbs/Gossiping/index.html'
    today = time.strftime("%m/%d").lstrip('0')
    date_range = [today]
    pptdata = []  
    num = 0

    # 第一頁 (含熱門文章)
    web_url = domain_url + Gossiping
    ppt_page = get_ppt_page(web_url)
    prev_href, pptdata, whether_prev, num = get_pageinfo(num, ppt_page, domain_url, pptdata, date_range) 
    
    # 往前一頁
    while whether_prev:
        web_url = domain_url+prev_href
        ppt_page = get_ppt_page(web_url)
        prev_href, pptdata, whether_prev, num = get_pageinfo(num, ppt_page, domain_url, pptdata, date_range) 
    
    # 將list轉成dataframe
    df = pd.DataFrame(pptdata, columns=['date','push_num', 'title', 'content', 'href'])  

    # 存成dataframe
      

In [16]:
date_ = date_range[0].replace("/", "_")

In [17]:
df.to_csv(date_+"_ppt_result.csv", index=False)